![A Cloud Guru](acg_logo.png)
<hr/>

<center><h1>Hyperparameter Tuning Job Created Using Amazon SageMaker</h1></center>

# Introduction

In this lab, you will learn how to import the iris dataset, split it into training and validation data, upload them to S3 bucket, fetch the lienar learner algorithm, initialize the estimator object, and automatically tune the hyperparameters using Amazon SageMaker's Automatic Model Tuning (AMT).

# How to Use This Lab

Most of the code is provided for you in this lab as our solution to the tasks presented. Some of the cells are left empty with a #TODO header and its your turn to fill in the empty code. You can always use our lab guide if you are stuck.

# 1) Install dependencies and import the required libraries

In [ ]:
# Install Sagemaker
!pip install boto3 sagemaker

1. We will use the iris dataset as our input data. 
2. The S3 bucket that you want to use for training data must be within the same region as the Notebook Instance.
3. The IAM role arn is used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

In [ ]:
import sagemaker
import boto3
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sagemaker.tuner import HyperparameterTuner, ContinuousParameter

# Initialize the SageMaker session
sagemaker_session = sagemaker.Session()

# Define the S3 bucket and prefix to store data
output_bucket = sagemaker.Session().default_bucket()
output_prefix = 'sagemaker/linear-learner'

In [ ]:
#TODO: Fetch the IAM role using the get_execution_role function and assign the value to a variable `role`

# 2) Download the data and upload them to S3 bucket

1. load_iris function is used to downlaod the input data
2. The data is split into training and validatin data in the ration of 80 - 20
3. The data is saved under 'train.csv' and 'validation.csv'

In [ ]:
# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Convert to DataFrame for easier manipulation
data = pd.DataFrame(X, columns=iris.feature_names)
data['target'] = y


In [ ]:
#TODO: Use the `train_test_split` function and the split the data in a 80 - 20 ratio. 
#TODO: Assign the values to variables `train_data` and `validation_data` 


In [ ]:
# Save to CSV
train_data.to_csv('train.csv', index=False, header=False)
validation_data.to_csv('validation.csv', index=False, header=False)

Let's use the upload_file function and upload the .csv files to the S3 buckets

In [ ]:
# Upload data to S3
s3 = boto3.client('s3')
s3.upload_file('train.csv', output_bucket, f'{output_prefix}/train/train.csv')

In [ ]:
#TODO: Using the strategy we followed to upload the training data as shown above, please upload the validation data to the output bucket.


# 3) Set up training and validation data

Create three separate variables that is dynamically constructed which will be used as one of the input parameters while generating training inputs

In [ ]:
# creating the inputs for the fit() function with the training and validation location
s3_train_data = f"s3://{output_bucket}/{output_prefix}/train"
print(f"training files will be taken from: {s3_train_data}")
s3_validation_data = f"s3://{output_bucket}/{output_prefix}/validation"
print(f"validation files will be taken from: {s3_validation_data}")
output_location = f"s3://{output_bucket}/{output_prefix}/output"
print(f"training artifacts output location: {output_location}")

Let's create the sagemaker.session.s3_input objects from our data channels. Note that we are using the content_type as text/csv. We use two channels here one for training and the second one for validation.

In [ ]:
# generating the session.s3_input() format for fit() accepted by the sdk
train_data = sagemaker.inputs.TrainingInput(
    s3_train_data,
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
    record_wrapping=None,
    compression=None,
)

In [ ]:
#TODO: Following the same strategy as shown above, please set up a training input for validation data.
#TODO: Name it as `validation_data`

# 4) Fetch the algorithm and initialize estimator

Let's retrieve the image for the Linear Learner Algorithm according to the region.

In [ ]:
# getting the linear learner image according to the region
from sagemaker.image_uris import retrieve

container = retrieve("linear-learner", boto3.Session().region_name, version="1")
print(container)
deploy_amt_model = True

Then we create an estimator from the SageMaker Python SDK using the Linear Learner container image and we setup the training parameters 

In [ ]:
%%time
import boto3
import sagemaker
from time import gmtime, strftime

sess = sagemaker.Session()

job_name = "linear-learner-iris-regression-" + strftime("%Y%m%d-%H-%M-%S", gmtime())
print("Training job", job_name)

linear = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type="ml.m5.large",
    output_path=output_location,
    sagemaker_session=sagemaker_session,
)

# 5) Define hyperparameter ranges and invoke tuning job

Set the initial values for the hyperparameters

In [ ]:
# TODO: Use the set_hyperparameters function and set the initial hyperparameters on linear learner
# feature_dim=4, predictor_type='regressor', mini_batch_size=20

Lets use the Continous parameter range and define the `learning rate` and `wd`

In [ ]:
#TODO: Define the hyperparameter ranges
#1. 'learning_rate': ContinuousParameter(0.01, 0.2)
#2. 'wd': ContinuousParameter(0.0, 0.1)

1. Instead of manually configuring our hyper parameter values and training with SageMaker Training, we'll use Amazon SageMaker Automatic Model Tuning.
2. The code sample below shows you how to use the HyperParameterTuner. It accepts the hyperparameter ranges we set previously.
3. Based on your capacity, you can adjust the `max_jobs` and `max_parallel_jobs`
4. The goal of the tuning job is to minimize `rmse`
5. The tuning job will take 8 to 10 minutes to complete.

In [ ]:
# Create a HyperparameterTuner object
tuner = HyperparameterTuner(
    estimator=linear,
    objective_metric_name='validation:rmse',
    hyperparameter_ranges=hyperparameter_ranges,
    metric_definitions=[
        {'Name': 'validation:rmse', 'Regex': 'validation rmse=([0-9\\.]+)'}
    ],
    max_jobs=4,
    max_parallel_jobs=2,
    objective_type='Minimize'
)

In [ ]:
#TODO: Initiate the tuner job by invoking the fit function.
#2. Pass the train_data and validation_data as input parameters